# Nodejs tensorflow model example

 * Wrap a nodejs tensorflow model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
 
## Depenencies

 * ```pip install seldon-core```
 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)
 * ```pip install grpcio-tools```
 * node (version>=8.11.0)
 * npm

## Train locally using npm commands
This model example takes an input of 10 different features and predicts a out for the same. For the training part it uses a random normally distributed input set of 100 rows i.e a data set of [100,10] and trains it for another random normally distributed data set of size [100,1]. For every prediction the model expects a dataset of dimension [r,10] where r is the num of input rows to be predicted.

In [ ]:
!make train && make clean_build

Training creates a model.json file and a weights.bin file which is utilized for prediction

## Prediction using REST API on the docker container

In [ ]:
!s2i build . seldonio/seldon-core-s2i-nodejs:0.2-SNAPSHOT node-s2i-model-image:0.1

In [ ]:
!docker run --name "nodejs_tensorflow_predictor" -d --rm -p 5000:5000 node-s2i-model-image:0.1

Send some random features that conform to the contract

In [ ]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p -t

In [ ]:
!docker rm nodejs_tensorflow_predictor --force

## Prediction using GRPC API on the docker container

In [ ]:
!s2i build -E ./.s2i/environment_grpc . seldonio/seldon-core-s2i-nodejs:0.2-SNAPSHOT node-s2i-model-image:0.2

In [ ]:
!docker run --name "nodejs_tensorflow_predictor" -d --rm -p 5000:5000 node-s2i-model-image:0.2

Send some random features that conform to the contract

In [ ]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p -t --grpc

In [ ]:
!docker rm nodejs_tensorflow_predictor --force

# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [ ]:
!minikube start --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

In [ ]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-nodejs:0.2-SNAPSHOT node-s2i-model-image:0.1

In [ ]:
!kubectl create -f nodejs_tensorflow_deployment.json

Wait until ready (replicas == replicasAvailable)

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' 

In [ ]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

In [ ]:
!minikube delete

In [ ]:
!make clean